In [3]:
import json

# Load the data from the "admin.jsonl" file
data = []
with open("admin.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

# Find points without a "cat" label
points_without_cat = []
for entry in data:
    if "cats" not in entry or not entry["cats"]:
        points_without_cat.append(entry)

# Print the points without a "cat" label
for entry in points_without_cat:
    print("Point ID:", entry["id"])
    print("Text:", entry["text"])
    print("Entities:", entry["entities"])
    print("Comments:", entry.get("Comments", []))  # Use .get() to handle missing "Comments" key
    print()


In [9]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load the data from the "admin.jsonl" file
data = []
with open("admin.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

# Separate features (email text and entities) and labels (cats) from the data
features = [entry["text"] for entry in data]
entities = [entry["entities"] for entry in data]
labels = [entry["cats"] for entry in data]

# Split the data into training (80%), validation (10%), and test (10%) sets
# First, split the data into training and the remaining data
X_train, X_remaining, y_train, y_remaining, entities_train, entities_remaining = train_test_split(
    features, labels, entities, test_size=0.2, random_state=42, stratify=labels
)

# Then, split the remaining data into validation and test sets (equal split)
X_val, X_test, y_val, y_test, entities_val, entities_test = train_test_split(
    X_remaining, y_remaining, entities_remaining, test_size=0.5, random_state=42, stratify=y_remaining
)

# Now you have the following splits:
# X_train, y_train, entities_train: Training set (80%)
# X_val, y_val, entities_val: Validation set (10%)
# X_test, y_test, entities_test: Test set (10%)

# Optionally, you can save these splits into separate files for future use
# For example:
with open("train.jsonl", "w") as file:
    for text, cats, entities in zip(X_train, y_train, entities_train):
        entry = {
            "text": text,
            "cats": cats,
            "entities": entities
        }
        file.write(json.dumps(entry) + "\n")

with open("val.jsonl", "w") as file:
    for text, cats, entities in zip(X_val, y_val, entities_val):
        entry = {
            "text": text,
            "cats": cats,
            "entities": entities
        }
        file.write(json.dumps(entry) + "\n")

with open("test.jsonl", "w") as file:
    for text, cats, entities in zip(X_test, y_test, entities_test):
        entry = {
            "text": text,
            "cats": cats,
            "entities": entities
        }
        file.write(json.dumps(entry) + "\n")


In [14]:
import json
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load the spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load the data from the "admin.jsonl" file
data = []
with open("admin.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

# Extract text, entities, and categories from the data
texts = [entry["text"] for entry in data]
entities = [entry["entities"] for entry in data]
categories = [entry["cats"][0] if entry["cats"] else "other" for entry in data]

# Extract labeled entities from the text using spaCy NER
extracted_entities = []
for text, ents in zip(texts, entities):
    doc = nlp(text)
    extracted_entities.append([ent.label_ for ent in doc.ents])

# Combine extracted entities with the original text
combined_texts = [" ".join([text] + entities) for text, entities in zip(texts, extracted_entities)]

# Encode categories into numerical values
category_mapping = {
    "report_request": 0,
    "email_list_edit": 1,
    "report_issue": 2,
    "report_cancel": 3,
    "other": 4
}
labels = [category_mapping[cat] for cat in categories]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(combined_texts, labels, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test_vectorized)
print(classification_report(y_test, y_pred, target_names=category_mapping.keys()))


                 precision    recall  f1-score   support

 report_request       0.81      0.57      0.67        53
email_list_edit       1.00      0.21      0.35        28
   report_issue       0.46      0.88      0.60        69
  report_cancel       1.00      0.25      0.40        12
          other       0.71      0.39      0.51        38

       accuracy                           0.57       200
      macro avg       0.80      0.46      0.51       200
   weighted avg       0.71      0.57      0.56       200



In [27]:
print(extracted_entities)

[['PERSON', 'DATE', 'ORG', 'DATE', 'TIME', 'DATE', 'TIME', 'DATE', 'ORG', 'PERSON', 'ORG', 'ORG', 'DATE'], ['PERSON', 'ORG', 'ORG', 'DATE', 'PERSON', 'CARDINAL', 'PERSON', 'DATE', 'DATE', 'PERSON', 'PERSON', 'PERSON', 'TIME', 'DATE', 'ORG', 'DATE', 'ORG'], ['PERSON', 'ORG', 'ORG', 'CARDINAL', 'GPE', 'ORG', 'ORG', 'GPE', 'GPE', 'ORG'], ['ORG', 'ORG', 'ORG', 'DATE', 'DATE', 'ORG', 'CARDINAL', 'GPE', 'ORG', 'ORG', 'GPE', 'GPE', 'ORG'], ['PERSON', 'PERSON', 'PRODUCT', 'CARDINAL', 'PERSON', 'PRODUCT', 'PERSON', 'DATE', 'TIME', 'PERSON', 'PERSON', 'ORG', 'PERSON', 'ORG'], ['GPE', 'LAW', 'DATE', 'PERSON', 'PRODUCT', 'CARDINAL', 'PERSON', 'PRODUCT', 'PERSON', 'DATE', 'TIME', 'PERSON', 'PERSON', 'ORG', 'PERSON', 'ORG'], ['TIME', 'ORG', 'PERSON', 'ORG', 'ORG', 'CARDINAL', 'CARDINAL', 'GPE', 'DATE', 'ORG'], ['DATE', 'ORG'], ['PERSON', 'ORG', 'DATE', 'ORG', 'CARDINAL', 'ORG', 'CARDINAL', 'GPE', 'DATE', 'ORG'], ['PERSON', 'ORG', 'GPE', 'ORG', 'CARDINAL', 'CARDINAL', 'PERSON', 'DATE', 'ORDINAL', 'PE

In [24]:
import json
import spacy
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Rest of the code remains the same

# Extract text, entities, and categories from the data
texts = [entry["text"] for entry in data]
entities = [entry["entities"] for entry in data]
categories = [entry["cats"][0] if entry["cats"] else "other" for entry in data]

# Function to replace characters within the entity spans with the entity label
def replace_entities_with_labels(text, entities):
    entities_map = {}
    for start, end, label in entities:
        entities_map[(start, end)] = label
    text_with_labels = ""
    current_position = 0
    for i, char in enumerate(text):
        if (current_position, i) in entities_map:
            text_with_labels += entities_map[(current_position, i)]
            current_position = i + 1
    text_with_labels += text[current_position:]
    return text_with_labels

# Combine extracted entities with the original text
combined_texts = [replace_entities_with_labels(text, entities_list) for text, entities_list in zip(texts, entities)]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(combined_texts, categories, test_size=0.2, random_state=42)

# Rest of the code remains the same
category_mapping = {
    "report_request": 0,
    "email_list_edit": 1,
    "report_issue": 2,
    "report_cancel": 3,
    "other": 4
}

# Convert categories to numerical values
y_train = np.array([category_mapping[label] for label in y_train])
y_test = np.array([category_mapping[label] for label in y_test])

# Rest of the code remains the same

# Convert texts to sequences using the Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1
# Pad sequences to have the same length
X_train_padded = pad_sequences(X_train_sequences, padding='post')
X_test_padded = pad_sequences(X_test_sequences, padding='post')
from keras.layers import LSTM
# Rest of the code remains the same
# Calculate the maximum sequence length
max_length = max(len(sequence) for sequence in X_train_padded)

# Create the neural network model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(category_mapping), activation='softmax'))

# Rest of the code remains the same


# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.1)
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10


2023-07-26 11:21:51.252801: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-26 11:21:51.254214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-26 11:21:51.255463: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

23/23 [==============================] - ETA: 0s - loss: 1.4832 - accuracy: 0.3833

2023-07-26 11:21:58.834077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-26 11:21:58.835391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-26 11:21:58.836618: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

23/23 [==============================] - 8s 290ms/step - loss: 1.4832 - accuracy: 0.3833 - val_loss: 1.4985 - val_accuracy: 0.3500
Epoch 2/10
23/23 [==============================] - 6s 275ms/step - loss: 1.4569 - accuracy: 0.3931 - val_loss: 1.5272 - val_accuracy: 0.3500
Epoch 3/10
23/23 [==============================] - 6s 277ms/step - loss: 1.4578 - accuracy: 0.3931 - val_loss: 1.4982 - val_accuracy: 0.3500
Epoch 4/10
23/23 [==============================] - 6s 276ms/step - loss: 1.4475 - accuracy: 0.3931 - val_loss: 1.4986 - val_accuracy: 0.3500
Epoch 5/10
23/23 [==============================] - 6s 277ms/step - loss: 1.4511 - accuracy: 0.3931 - val_loss: 1.5004 - val_accuracy: 0.3500
Epoch 6/10
23/23 [==============================] - 6s 276ms/step - loss: 1.4487 - accuracy: 0.3931 - val_loss: 1.5080 - val_accuracy: 0.3500
Epoch 7/10
23/23 [==============================] - 6s 276ms/step - loss: 1.4484 - accuracy: 0.3931 - val_loss: 1.5069 - val_accuracy: 0.3500
Epoch 8/10
23/23 

2023-07-26 11:22:56.460233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-26 11:22:56.461498: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-26 11:22:56.462770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7/7 [==============================] - 1s 86ms/step - loss: 1.4966 - accuracy: 0.3450
Test Loss: 1.4966347217559814
Test Accuracy: 0.3449999988079071


In [40]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Load data from admin.jsonl
data = []
with open("admin.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        entry = json.loads(line)
        text = entry["text"]
        entities = entry.get("entities", [])
        category = entry["cats"][0] if entry["cats"] else "other"

        # Extract entities and combine them with the original text
        entities_text = " ".join([text[s:e] for s, e, _ in entities])
        combined_text = text + " " + entities_text

        data.append({"text": combined_text, "category": category})

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Define the category mapping
category_mapping = {
    "report_request": 0,
    "email_list_edit": 1,
    "report_issue": 2,
    "report_cancel": 3,
    "other": 4
}

# Map the categories to integers
df["category"] = df["category"].map(category_mapping)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["category"], test_size=0.2, random_state=42)

# Vectorize the combined texts using TF-IDF or CountVectorizer
vectorizer = TfidfVectorizer()  # You can also use CountVectorizer if you prefer
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vectorized)
print(classification_report(y_test, y_pred, target_names=category_mapping.keys()))

# Save the trained model to a file
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(model, 'logistic_regression_model.pkl')


                 precision    recall  f1-score   support

 report_request       0.89      0.64      0.75        53
email_list_edit       0.90      0.32      0.47        28
   report_issue       0.54      0.96      0.69        69
  report_cancel       1.00      0.33      0.50        12
          other       0.81      0.55      0.66        38

       accuracy                           0.67       200
      macro avg       0.83      0.56      0.61       200
   weighted avg       0.76      0.67      0.66       200



['logistic_regression_model.pkl']

In [85]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import classification_report
import joblib

# Load the data from admin.jsonl
data = []
with open("admin.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        entry = json.loads(line)
        data.append(entry)

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Extract the entity labels
def extract_entity_labels(entities):
    return [entity[2] for entity in entities]

df['entity_labels'] = df['entities'].apply(extract_entity_labels)

# Combine the identified entities with the original text
df['combined_text'] = df.apply(lambda row: " ".join([row['text'][start:end] for start, end, _ in row['entities']]), axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df["combined_text"], df["entity_labels"], test_size=0.2, random_state=42)

# Train the Entity Classification Model
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

mlb = MultiLabelBinarizer()
y_train_transformed = mlb.fit_transform(y_train)
y_test_transformed = mlb.transform(y_test)

base_classifier = LogisticRegression(max_iter=1000)
multi_label_classifier = MultiOutputClassifier(base_classifier)
multi_label_classifier.fit(X_train_vectorized, y_train_transformed)

# Convert the "cats" field into a flat list of category labels
category_labels = [label for sublist in df["cats"] for label in sublist]

# Train the Email Category Classification Model
category_vectorizer = TfidfVectorizer()
X_train_category_vectorized = category_vectorizer.fit_transform(df["text"])
category_encoder = LabelEncoder()
y_train_category_encoded = category_encoder.fit_transform(category_labels)

category_model = LogisticRegression(max_iter=1000)
category_model.fit(X_train_category_vectorized, y_train_category_encoded)

# Save the trained models to files
joblib.dump(vectorizer, "entity_vectorizer.pkl")
joblib.dump(multi_label_classifier, "entity_classification_model.pkl")
joblib.dump(category_vectorizer, "category_vectorizer.pkl")
joblib.dump(category_encoder, "category_encoder.pkl")
joblib.dump(category_model, "category_classification_model.pkl")

# Sample text to test the model
sample_text = "Please add john@example.com and jane@example.com to the mailing list and remove sarah@example.com."

# Use the entity classification model to predict entities in the sample text
sample_text_vectorized = vectorizer.transform([sample_text])
predicted_labels = multi_label_classifier.predict(sample_text_vectorized)
predicted_entities = mlb.inverse_transform(predicted_labels)

# Use the predicted entities to predict the email category
sample_text_category_vectorized = category_vectorizer.transform([sample_text])
predicted_category_encoded = category_model.predict(sample_text_category_vectorized)
predicted_category_encoded = predicted_category_encoded.reshape(-1, 1)
predicted_category = category_encoder.inverse_transform(predicted_category_encoded)

print("Sample Text:", sample_text)
print("Predicted Entities:", predicted_entities)
print("Predicted Email Category:", predicted_category)

# Classification Report for Entity Classification Model
y_pred_entities = multi_label_classifier.predict(X_test_vectorized)
target_names_entities = mlb.classes_
print("Classification Report for Entity Classification Model:")
print(classification_report(y_test_transformed, y_pred_entities, target_names=target_names_entities))

# Classification Report for Email Category Classification Model
y_pred_category = category_model.predict(X_train_category_vectorized)
target_names_category = category_encoder.classes_
print("Classification Report for Email Category Classification Model:")
print(classification_report(y_train_category_encoded, y_pred_category, target_names=target_names_category))


Sample Text: Please add john@example.com and jane@example.com to the mailing list and remove sarah@example.com.
Predicted Entities: [('add_emails', 'emails_to_add')]
Predicted Email Category: ['email_list_edit']
Classification Report for Entity Classification Model:
                  precision    recall  f1-score   support

      add_emails       0.89      0.35      0.50        23
      add_fields       0.93      0.39      0.55        33
       attention       0.00      0.00      0.00         0
   change_fields       0.00      0.00      0.00         1
          client       0.92      0.83      0.87       110
   create_report       0.90      0.42      0.58        45
   emails_to_add       0.80      0.24      0.36        17
emails_to_remove       0.00      0.00      0.00         7
   fields_to_add       1.00      0.44      0.61        25
fields_to_change       0.00      0.00      0.00         0
fields_to_remove       0.00      0.00      0.00         1
  issue_to_check       0.82      0.9

/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py

In [83]:
# Sample text to test the model
sample_text = "Please add john@example.com and jane@example.com to the mailing list and remove sarah@example.com."

# Use the entity classification model to predict entities in the sample text
sample_text_vectorized = vectorizer.transform([sample_text])
predicted_labels = multi_label_classifier.predict(sample_text_vectorized)
predicted_entities = mlb.inverse_transform(predicted_labels)

# Use the predicted entities to predict the email category
sample_text_category_vectorized = category_vectorizer.transform([sample_text])
predicted_category_encoded = category_model.predict(sample_text_category_vectorized)
predicted_category_encoded = predicted_category_encoded.reshape(-1, 1)
predicted_category = category_encoder.inverse_transform(predicted_category_encoded)

print("Sample Text:", sample_text)

# Print Predicted Entities
print("Predicted Entities:")
found_entities = False
for entity_type, entity_text in zip(mlb.classes_, predicted_entities[0]):
    for start, end in zip(*np.where(sample_text_vectorized.toarray()[:, vectorizer.vocabulary_[entity_text]] > 0)):
        print(f"{entity_type}: {entity_text} (Character Range: {start}-{end})")
        found_entities = True

if not found_entities:
    print("No entities identified in the sample text.")

print("Predicted Email Category:", predicted_category[0])


Sample Text: Please add john@example.com and jane@example.com to the mailing list and remove sarah@example.com.
Predicted Entities:


/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyError: 'add_emails'

In [41]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained models and category mapping
entity_classification_model = joblib.load('entity_classification_model.pkl')
model = joblib.load('logistic_regression_model.pkl')
category_mapping = {
    "report_request": 0,
    "email_list_edit": 1,
    "report_issue": 2,
    "report_cancel": 3,
    "other": 4
}

# Sample text to test the model
sample_text = "can we remove kaydenlea@gmail.com from this distribution from now on. "

# Step 1: Use the entity classification model to identify entities in the sample text
identified_entities = entity_classification_model.predict([sample_text])

# Step 2: Combine identified entities with the sample text
entities_text = " ".join([sample_text[start:end] for start, end, _ in identified_entities])
combined_text = sample_text + " " + entities_text

# Step 3: Load the vectorizer used during training and vectorize the combined text
vectorizer = joblib.load('tfidf_vectorizer.pkl')
sample_text_vectorized = vectorizer.transform([combined_text])

# Step 4: Make predictions on the sample text using the logistic regression model
predicted_category = model.predict(sample_text_vectorized)[0]
predicted_category_label = [k for k, v in category_mapping.items() if v == predicted_category][0]

print(f"Predicted Category: {predicted_category_label}")


ValueError: Expected 2D array, got 1D array instead:
array=['can we remove kaydenlea@gmail.com from this distribution from now on. '].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [21]:
import json

# Load the data from admin.jsonl
data = []
with open("admin.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        data.append(json.loads(line))

# Create a list to store the extracted entities
entities_data = []

# Extract the entities and their corresponding entity labels
for _, row in enumerate(data):
    text = row["text"]
    entities = row["entities"]
    for start, end, entity_label in entities:
        entity_text = text[start:end]
        entities_data.append({"text": entity_text, "category": entity_label})

# Save the entity data to a separate file (e.g., entities_data.jsonl)
with open("entities_data.jsonl", "w", encoding="utf-8") as output_file:
    for entity in entities_data:
        output_file.write(json.dumps(entity, ensure_ascii=False) + "\n")


In [72]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import joblib

# Load the data from admin.jsonl (Assuming you have admin.jsonl file)
data = []
with open("admin.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        entry = json.loads(line)
        data.append(entry)

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Extract the entity labels
def extract_entity_labels(entities):
    return [entity[2] for entity in entities]

df['entity_labels'] = df['entities'].apply(extract_entity_labels)

# Combine the identified entities with the original text
df['combined_text'] = df.apply(lambda row: " ".join([row['text'][start:end] for start, end, _ in row['entities']]), axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df["combined_text"], df["entity_labels"], test_size=0.2, random_state=42)

# Convert the entity labels into a multi-label binary array
mlb = MultiLabelBinarizer()
y_train_transformed = mlb.fit_transform(y_train)
y_test_transformed = mlb.transform(y_test)

# Create a TfidfVectorizer to convert text data to numerical vectors
vectorizer = TfidfVectorizer()

# Vectorize the training and testing data
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the MultiOutputClassifier with Logistic Regression as the base estimator
base_classifier = LogisticRegression(max_iter=1000)
multi_label_classifier = MultiOutputClassifier(base_classifier)
multi_label_classifier.fit(X_train_vectorized, y_train_transformed)

# Save the trained model to a file
joblib.dump(vectorizer, "entity_vectorizer.pkl")
joblib.dump(multi_label_classifier, "entity_classification_model.pkl")

# Evaluate the model and print the classification report
y_pred_transformed = multi_label_classifier.predict(X_test_vectorized)
classification_rep = classification_report(y_test_transformed, y_pred_transformed, target_names=mlb.classes_)
print("Classification Report:")
print(classification_rep)

# Sample text to test the model
sample_text = "Please add john@example.com and jane@example.com to the mailing list and remove sarah@example.com."

# Vectorize the sample text using the same vectorizer used during training
sample_text_vectorized = vectorizer.transform([sample_text])

# Make predictions on the sample text
predicted_labels = multi_label_classifier.predict(sample_text_vectorized)
predicted_entities = mlb.inverse_transform(predicted_labels)

print("Sample Text:", sample_text)
print("Predicted Entities:")
for entity_type, entity_text in zip(mlb.classes_, predicted_entities[0]):
    for entity in df.loc[df["combined_text"].str.contains(entity_text), "entities"].values[0]:
        if entity[2] == entity_type:
            print(f"{entity_type}: {entity_text} (Character Range: {entity[0]}-{entity[1]})")
            break


Classification Report:
                  precision    recall  f1-score   support

      add_emails       0.89      0.35      0.50        23
      add_fields       0.93      0.39      0.55        33
       attention       0.00      0.00      0.00         0
   change_fields       0.00      0.00      0.00         1
          client       0.92      0.83      0.87       110
   create_report       0.90      0.42      0.58        45
   emails_to_add       0.80      0.24      0.36        17
emails_to_remove       0.00      0.00      0.00         7
   fields_to_add       1.00      0.44      0.61        25
fields_to_change       0.00      0.00      0.00         0
fields_to_remove       0.00      0.00      0.00         1
  issue_to_check       0.82      0.93      0.87        96
   remove_emails       1.00      0.16      0.27        19
   remove_fields       0.00      0.00      0.00         1
   remove_report       1.00      0.25      0.40        12
          urgent       1.00      0.04      0.08 

/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaydenlea/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaydenlea/.local/lib/python3.8/si

IndexError: index 0 is out of bounds for axis 0 with size 0